In [2]:
import numpy as np 
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder , StandardScaler, OrdinalEncoder 
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, LinearRegression


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

In [3]:
df = pd.read_csv('gurgaon_properties_post_feature_selection_v2.csv')

In [4]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 7,0.45,2,2,1,Relatively New,814.0,0,0,1,Low,Mid Floor
1,flat,sector 3,0.50,2,2,1,Old Property,588.0,0,0,0,Low,Low Floor
2,flat,sohna road,0.40,2,2,3,New Property,538.0,0,0,1,Low,High Floor
3,flat,sector 61,1.47,2,2,2,New Property,1086.0,0,0,1,Medium,Low Floor
4,flat,sector 92,0.70,2,2,3,Under Construction,1217.0,0,0,1,Low,Mid Floor


In [5]:
df['furnishing_type'] = df['furnishing_type'].replace({0:'unfurnished' , 1:'semifurnished', 2:'furnished'})

In [6]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 7,0.45,2,2,1,Relatively New,814.0,0,0,semifurnished,Low,Mid Floor
1,flat,sector 3,0.50,2,2,1,Old Property,588.0,0,0,unfurnished,Low,Low Floor
2,flat,sohna road,0.40,2,2,3,New Property,538.0,0,0,semifurnished,Low,High Floor
3,flat,sector 61,1.47,2,2,2,New Property,1086.0,0,0,semifurnished,Medium,Low Floor
4,flat,sector 92,0.70,2,2,3,Under Construction,1217.0,0,0,semifurnished,Low,Mid Floor


In [7]:
x = df.drop(columns = ['price'])
y = df['price']

In [8]:
y_transformed = np.log1p(y)

In [9]:
columns_to_encode = ['property_type','sector','balcony','agePossession','furnishing_type','luxury_category','floor_category','servant room','store room']

In [10]:
preprocessor = ColumnTransformer(
transformers = [
    ('num',StandardScaler() , ['bedRoom','bathroom','built_up_area']),
    ('cat',OrdinalEncoder() , columns_to_encode)
],
remainder = 'passthrough')

In [11]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
    
])

In [12]:
kfold = KFold(n_splits = 10 , shuffle = True , random_state = 42)
scores = cross_val_score(pipeline , x, y_transformed, cv = kfold , scoring = 'r2')

In [13]:
scores.mean(),scores.std()

(np.float64(0.7380880150557887), np.float64(0.025397705027738922))

In [14]:
x_train,x_test,y_train,y_test = train_test_split(x, y_transformed , test_size = .2 , random_state = 42 )

In [15]:
pipeline.fit(x_train,y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [16]:
y_pred = pipeline.predict(x_test)

In [17]:
y_pred = np.expm1(y_pred)

In [18]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.9444749185138055

In [19]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    x_train, x_test, y_train, y_test = train_test_split(x,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(x_train,y_train)
    
    y_pred = pipeline.predict(x_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [20]:
!pip install xgboost

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor
)
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor


In [21]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [22]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))
model_output

C:\Users\SWAPN\OneDrive\Desktop\Envs\base_env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


[['linear_reg', np.float64(0.7380880150557887), 0.9444749185138055],
 ['svr', np.float64(0.7660903346701), 0.896399710936858],
 ['ridge', np.float64(0.7380924748351866), 0.9443610248087742],
 ['LASSO', np.float64(0.05461637521819442), 1.6085825499224644],
 ['decision tree', np.float64(0.7942637039251963), 0.720115356463631],
 ['random forest', np.float64(0.8864770849726857), 0.546432340529027],
 ['extra trees', np.float64(0.8773154840506201), 0.6138852837385509],
 ['gradient boosting', np.float64(0.8786771873198347), 0.5809541607617493],
 ['adaboost', np.float64(0.7605377584907979), 0.8705272471591672],
 ['mlp', np.float64(0.8195962158085199), 0.6915067868305381],
 ['xgboost', np.float64(0.8919683010338432), 0.5616582050571629]]

In [23]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.886477,0.546432
10,xgboost,0.891968,0.561658
7,gradient boosting,0.878677,0.580954
6,extra trees,0.877315,0.613885
9,mlp,0.819596,0.691507
4,decision tree,0.794264,0.720115
8,adaboost,0.760538,0.870527
1,svr,0.766090,0.896400
2,ridge,0.738092,0.944361
0,linear_reg,0.738088,0.944475


In [24]:
df['servant room'].value_counts()
df['store room'].value_counts()

store room
0    3233
1     321
Name: count, dtype: int64

In [25]:
columns_to_encode = ['property_type','balcony','servant room','store room','furnishing_type','floor_category','luxury_category']

In [26]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [27]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')
scores.mean()

np.float64(0.8581008503127064)

In [28]:
scores.std()

np.float64(0.021138311846273523)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(x,y_transformed,test_size=0.2,random_state=42)
pipeline.fit(X_train,y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [30]:
y_pred = pipeline.predict(X_test)
y_pred = np.expm1(y_pred)
mean_absolute_error(np.expm1(y_test),y_pred)

0.6893876970914693

In [31]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(x,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output

In [32]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [33]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [34]:
model_df.sort_values(['mae'])

,name,r2,mae
6,extra trees,0.888663,0.559482
10,xgboost,0.894963,0.560868
1,svr,0.887110,0.566454
9,mlp,0.876061,0.569662
5,random forest,0.875337,0.597553
7,gradient boosting,0.861791,0.640871
0,linear_reg,0.858101,0.689388
2,ridge,0.856090,0.695892
4,decision tree,0.791251,0.737476
8,adaboost,0.735134,0.919943


In [35]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [36]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.95)),
    ('regressor', LinearRegression())
])

In [37]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')
scores.mean()

np.float64(0.7632635784000927)

In [38]:
scores.std()

np.float64(0.02396135995645302)

In [39]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(x,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [40]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])
model_df.sort_values(['mae'])

C:\Users\SWAPN\OneDrive\Desktop\Envs\base_env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,name,r2,mae
6,extra trees,0.842998,0.676999
10,xgboost,0.834611,0.679682
9,mlp,0.819989,0.683242
5,random forest,0.827836,0.704049
7,gradient boosting,0.822840,0.728524
1,svr,0.824483,0.730128
0,linear_reg,0.763264,0.872512
2,ridge,0.763340,0.872863
4,decision tree,0.642107,0.955830
8,adaboost,0.689591,0.981071


In [42]:
import category_encoders as ce

columns_to_encode = ['property_type','balcony','servant room','store room','furnishing_type','floor_category','luxury_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)
!pip install category_encoders
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')
scores.mean(),scores.std()

(np.float64(0.830979398693217), np.float64(0.02059123713733806))

In [43]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(x,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [45]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.898986,0.544818
10,xgboost,0.896363,0.548084
6,extra trees,0.898458,0.554348
7,gradient boosting,0.888711,0.584778
1,svr,0.868935,0.629106
9,mlp,0.853947,0.698910
4,decision tree,0.811781,0.746839
0,linear_reg,0.830979,0.754033
2,ridge,0.831000,0.754314
8,adaboost,0.816468,0.755664


In [44]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__max_samples':[0.1, 0.25, 0.5, 1.0],
    'regressor__max_features': ['auto', 'sqrt']
}
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)
search.fit(x, y_transformed)

Fitting 10 folds for each of 128 candidates, totalling 1280 fits


C:\Users\SWAPN\OneDrive\Desktop\Envs\base_env\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
640 fits failed out of a total of 1280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
344 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SWAPN\OneDrive\Desktop\Envs\base_env\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SWAPN\OneDrive\Desktop\Envs\base_env\Lib\site-packages\sklearn\base.py", line 1363, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\SWAPN\OneDrive\Desktop\Envs\base_en

,estimator,Pipeline(step...Regressor())])
,param_grid,"{'regressor__max_depth': [None, 10, ...], 'regressor__max_features': ['auto', 'sqrt'], 'regressor__max_samples': [0.1, 0.25, ...], 'regressor__n_estimators': [50, 100, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,4
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...), ...]"


In [46]:
final_pipe = search.best_estimator_
search.best_params_

{'regressor__max_depth': 20,
 'regressor__max_features': 'sqrt',
 'regressor__max_samples': 1.0,
 'regressor__n_estimators': 300}

In [47]:
search.best_score_

np.float64(0.9038114185582113)

In [48]:
final_pipe.fit(x,y_transformed)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


## Exporting Model

In [49]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=500))
])

pipeline.fit(x,y_transformed)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [50]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)
with open('df.pkl', 'wb') as file:
    pickle.dump(x, file)
x

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 7,2,2,1,Relatively New,814.0,0,0,semifurnished,Low,Mid Floor
1,flat,sector 3,2,2,1,Old Property,588.0,0,0,unfurnished,Low,Low Floor
2,flat,sohna road,2,2,3,New Property,538.0,0,0,semifurnished,Low,High Floor
3,flat,sector 61,2,2,2,New Property,1086.0,0,0,semifurnished,Medium,Low Floor
4,flat,sector 92,2,2,3,Under Construction,1217.0,0,0,semifurnished,Low,Mid Floor
...,...,...,...,...,...,...,...,...,...,...,...,...
3549,house,sector 57,3,3,3,Moderately Old,1650.0,0,1,unfurnished,Medium,Low Floor
3550,house,sector 26,4,4,3,Moderately Old,1800.0,1,0,semifurnished,Medium,Low Floor
3551,house,sector 25,3,2,3,Old Property,1350.0,0,0,semifurnished,Low,Low Floor
3552,house,sector 26,3,3,2,Moderately Old,1350.0,1,0,semifurnished,Medium,Low Floor


In [51]:
x.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')

In [52]:
x.iloc[0].values

array(['flat', 'sector 7', np.int64(2), np.int64(2), '1',
       'Relatively New', np.float64(814.0), np.int64(0), np.int64(0),
       'semifurnished', 'Low', 'Mid Floor'], dtype=object)

In [53]:
data = [['house', 'sector 102', 4, 3, '3+', 'New Property', 2750, 0, 0, 'unfurnished', 'Low', 'Low Floor']]
columns = ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,house,sector 102,4,3,3+,New Property,2750,0,0,unfurnished,Low,Low Floor


In [54]:
np.expm1(pipeline.predict(one_df))

array([3.23569016])

In [55]:
x.dtypes

property_type       object
sector              object
bedRoom              int64
bathroom             int64
balcony             object
agePossession       object
built_up_area      float64
servant room         int64
store room           int64
furnishing_type     object
luxury_category     object
floor_category      object
dtype: object

In [56]:
sorted(x['sector'].unique().tolist())

['sector 1',
 'sector 10',
 'sector 102',
 'sector 103',
 'sector 104',
 'sector 105',
 'sector 106',
 'sector 107',
 'sector 108',
 'sector 109',
 'sector 11',
 'sector 110',
 'sector 111',
 'sector 112',
 'sector 113',
 'sector 12',
 'sector 13',
 'sector 14',
 'sector 15',
 'sector 17',
 'sector 2',
 'sector 21',
 'sector 22',
 'sector 23',
 'sector 24',
 'sector 25',
 'sector 26',
 'sector 27',
 'sector 28',
 'sector 3',
 'sector 30',
 'sector 31',
 'sector 33',
 'sector 36',
 'sector 37',
 'sector 38',
 'sector 39',
 'sector 4',
 'sector 40',
 'sector 41',
 'sector 43',
 'sector 45',
 'sector 46',
 'sector 47',
 'sector 48',
 'sector 49',
 'sector 5',
 'sector 50',
 'sector 51',
 'sector 52',
 'sector 53',
 'sector 54',
 'sector 55',
 'sector 56',
 'sector 57',
 'sector 58',
 'sector 59',
 'sector 6',
 'sector 60',
 'sector 61',
 'sector 62',
 'sector 63',
 'sector 65',
 'sector 66',
 'sector 67',
 'sector 68',
 'sector 69',
 'sector 7',
 'sector 70',
 'sector 70a',
 'sector 71',


In [60]:
sorted(x['sector'].unique().tolist())

['sector 1',
 'sector 10',
 'sector 102',
 'sector 103',
 'sector 104',
 'sector 105',
 'sector 106',
 'sector 107',
 'sector 108',
 'sector 109',
 'sector 11',
 'sector 110',
 'sector 111',
 'sector 112',
 'sector 113',
 'sector 12',
 'sector 13',
 'sector 14',
 'sector 15',
 'sector 17',
 'sector 2',
 'sector 21',
 'sector 22',
 'sector 23',
 'sector 24',
 'sector 25',
 'sector 26',
 'sector 27',
 'sector 28',
 'sector 3',
 'sector 30',
 'sector 31',
 'sector 33',
 'sector 36',
 'sector 37',
 'sector 38',
 'sector 39',
 'sector 4',
 'sector 40',
 'sector 41',
 'sector 43',
 'sector 45',
 'sector 46',
 'sector 47',
 'sector 48',
 'sector 49',
 'sector 5',
 'sector 50',
 'sector 51',
 'sector 52',
 'sector 53',
 'sector 54',
 'sector 55',
 'sector 56',
 'sector 57',
 'sector 58',
 'sector 59',
 'sector 6',
 'sector 60',
 'sector 61',
 'sector 62',
 'sector 63',
 'sector 65',
 'sector 66',
 'sector 67',
 'sector 68',
 'sector 69',
 'sector 7',
 'sector 70',
 'sector 70a',
 'sector 71',


In [63]:
x.dtypes

property_type       object
sector              object
bedRoom              int64
bathroom             int64
balcony             object
agePossession       object
built_up_area      float64
servant room         int64
store room           int64
furnishing_type     object
luxury_category     object
floor_category      object
dtype: object

In [64]:
sorted(df['bedRoom'].unique().tolist())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]